In [1]:
from visualizer import get_local
get_local.activate()

import os
ROOT_PATH="/data/aofei"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ["TRANSFORMERS_CACHE"] = f"{ROOT_PATH}/huggingface_cache/transformers"
os.environ["HF_HOME"] = f"{ROOT_PATH}/huggingface_cache/transformers"
os.environ["TORCH_HOME"] = f"{ROOT_PATH}/huggingface_cache/torch"


from PIL import Image
# import seaborn as sns
import torch
import torch.nn.functional as F
import json
import copy
import numpy as np

import sys
sys.path.append("/home/avc6555/research/MedH/Mitigation/LVLMs/XrayGPT")


import numpy as np
import torch
import torch.backends.cudnn as cudnn

from xraygpt.common.config import Config
from xraygpt.common.dist_utils import get_rank
from xraygpt.common.registry import registry
from xraygpt.conversation.conversation import Chat, CONV_VISION

# imports modules for registration
from xraygpt.datasets.builders import *
from xraygpt.models import *
from xraygpt.processors import *
from xraygpt.runners import *
from xraygpt.tasks import *

from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False


/data/aofei/conda/env/minigptv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_id = os.environ["CUDA_VISIBLE_DEVICES"]
class Args:
    def __init__(self):
        self.cfg_path="/home/avc6555/research/MedH/Mitigation/LVLMs/XrayGPT/eval_configs/xraygpt_eval.yaml"
        self.options=[]

args = Args()

cfg = Config(args)
model_config = cfg.model_cfg
model_config.device_8bit = gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda')
device = torch.device('cuda')
# CONV_VISION = conv_dict[model_config.model_type]

vis_processor_cfg = cfg.datasets_cfg.openi.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)

stop_words_ids = [torch.tensor([835]).to(device),
                    torch.tensor([2277, 29937]).to(device)]  # '###' can be encoded in two different ways.
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

prompt = conv.get_prompt()
prompt_segs = prompt.split('<ImageHere>')
seg_tokens = [
    model.llama_tokenizer(
        seg, return_tensors="pt", add_special_tokens=i == 0).to(model.device).input_ids
    # only add bos to the first seg
    for i, seg in enumerate(prompt_segs)
]
[i.size() for i in seg_tokens]

Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA


Loading checkpoint shards: 100%|██████████| 2/2 [01:13<00:00, 36.75s/it]


Loading LLAMA Done


FileNotFoundError: [Errno 2] No such file or directory: 'prompts/alignment.txt'

In [ ]:
def get_context_emb(conv, img_list, model):
    prompt = conv.get_prompt()
    prompt_segs = prompt.split('<ImageHere>')
    seg_tokens = [
        model.llama_tokenizer(
            seg, return_tensors="pt", add_special_tokens=i == 0).to(model.device).input_ids
        # only add bos to the first seg
        for i, seg in enumerate(prompt_segs)
    ]
    seg_embs = [model.llama_model.model.embed_tokens(seg_t) for seg_t in seg_tokens]
    mixed_embs = [emb for pair in zip(seg_embs[:-1], img_list) for emb in pair] + [seg_embs[-1]]
    mixed_embs = torch.cat(mixed_embs, dim=1)
    return mixed_embs

image = "/home/avc6555/research/MedH/Mitigation/LVLMs/llava-med/visualize/images/cat.jpg"
question = "What is the abnormity in this image?"
# question = "What are the main findings and impression of the given x-ray?"
conv = CONV_VISION

if isinstance(image, str):  # is a image path
    raw_image = Image.open(image).convert('RGB')
    image = vis_processor(raw_image).unsqueeze(0).to(device)
elif isinstance(image, Image.Image):
    raw_image = image
    image = vis_processor(raw_image).unsqueeze(0).to(device)
elif isinstance(image, torch.Tensor):
    if len(image.shape) == 3:
        image = image.unsqueeze(0)
    image = image.to(device)
img_list = []
image_emb, _ = model.encode_img(image)
img_list.append(image_emb)
conv.messages = []
conv.append_message(conv.roles[0], "<Img><ImageHere></Img>")
# question = "Could you highlight any abnormalities or concerns in this chest x-ray image?"
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
embs = get_context_emb(conv, img_list, model)

begin_idx = 0

embs = embs[:, begin_idx:]

outputs = model.llama_model.generate(
        inputs_embeds=embs,
        max_new_tokens=64,
        stopping_criteria=stopping_criteria,
        num_beams=1,
        do_sample=True,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1,
        length_penalty=1,
        temperature=1,
        output_attentions=True, return_dict_in_generate=True
    )
output_token = outputs['sequences'][0]
if output_token[0] == 0:  # the model might output a unknow token <unk> at the beginning. remove it
    output_token = output_token[1:]
if output_token[0] == 1:  # some users find that there is a start token <s> at the beginning. remove it
    output_token = output_token[1:]
output_text = model.llama_tokenizer.decode(output_token, add_special_tokens=False)
output_text = output_text.split('###')[0]  # remove the stop sign '###'
output_text = output_text.split('Doctor:')[-1].strip()